In [ ]:
import glob
from datetime import datetime
import os
from multiprocessing import Pool

import numpy as np

from lmsiage.uncertainty import ComputeIntegratedUncertainty
from lmsiage.mesh_file import MeshFile
from lmsiage.zarr_index_tools import cleanup_missing_files, files_with_array
cleanup_missing_files()

In [ ]:
n_steps = 2210  # should be 2210 to cover 6 years
sia_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/uncert'
age_dir = f'{sia_dir}/age'

unc_files = sorted(list(set(files_with_array('unc_sic') + files_with_array('unc_sid'))))
unc_dates = [datetime.strptime(f.split('/')[-1], 'unc_%Y%m%d.zip') for f in unc_files]
start_indices = [i for i in range(0, len(unc_dates)) if unc_dates[i].month == 9 and unc_dates[i].day == 15]
start_indices

In [ ]:
ciu = ComputeIntegratedUncertainty(mesh_dir, age_dir, unc_dir, n_steps)
for start_idx in start_indices:
    ciu.compute(unc_dates, start_idx)


In [ ]:
year = 1993
print(MeshFile(f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/{year}/unc_{year}1231.zip').read_names())

import matplotlib.pyplot as plt
unc_sic, unc_sid = MeshFile(f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/{year}/unc_{year}1231.zip').load(['unc_sic1991', 'unc_sid1991'], as_dict=False)
x, y, t = MeshFile(f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh/{year}/mesh_{year}1231.zip').load(['x', 'y', 't'], as_dict=False)
plt.tripcolor(x, y, t, unc_sid)
plt.colorbar()